In [15]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from datetime import datetime
import pandas as pd

now = datetime.now().time()
print("Start Time =", now)

HashData=[]
TimeData=[]
From=[]
To=[]
Value=[]
TimeStamp=[]
TransactionFee=[]
GasPrice=[]
Block=[]

opt=webdriver.ChromeOptions()
opt.add_argument("--headless")
user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.54 Safari/537.36'
opt.add_argument('user-agent={0}'.format(user_agent))

s=Service("C:\Program Files (x86)\chromedriver.exe")
driver= webdriver.Chrome(service=s,options=opt)

Start_Block=input("ENTER STARTING BLOCK NUMBER ")  #Block_Number="14800000"

TotalBlocks =int(input("Total Blocks To Fetch"))

for i in range(TotalBlocks):

    Block_Number=str(i+int(Start_Block))
    
    Block_Page_url= "https://etherscan.io/block/"+ Block_Number

    Txs_Page_url="https://etherscan.io/txs?block="+ Block_Number

    driver.get(Txs_Page_url)

    Total_txs_Info = driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div/div/div[2]/p/span")

    Total_txs= int(Total_txs_Info.text.split()[3])

    print("Total transaction in the block" , Total_txs)

    Pages= int(Total_txs/50)+1 if Total_txs%50 else int(Total_txs/50)

    print("Total Pages" , Pages)

    Current_Page_Txs = driver.find_elements(By.CLASS_NAME,"myFnExpandBox_searchVal")
    
    AllHashes=[x.text for x in Current_Page_Txs]
        

    MissingVal=0   
    
    for x in AllHashes:
        Hash=x
        Txs_url="https://etherscan.io/tx/"+ Hash
        driver.get(Txs_url)
        try:
            time_element=driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[4]/div/div[2]/span[2]").text
        except :
            MissingVal=MissingVal+1
        else:
            HashData.append(Hash)
            To.append(driver.find_element(By.ID,"contractCopy").text)
            From.append(driver.find_element(By.ID,"addressCopy").text)
            Value.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanValue").text)
            TimeStamp.append(driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[4]/div/div[2]").text)
            TransactionFee.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanTxFee").text)
            GasPrice.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanGasPrice").text)
            TimeData.append(time_element)    
                
        #print(HashData[-1]+"  "+TimeData[-1]+"  "+To[-1]+"  "+From[-1]+"  "+Value[-1]+"  "+TimeStamp[-1]+"  "+TransactionFee[-1]+"  "+GasPrice[-1]) 

        
    for i in range(Pages-1):
        driver.get(Txs_Page_url+"&p="+str(i+2))

        Current_Page_Txs = driver.find_elements(By.CLASS_NAME,"myFnExpandBox_searchVal")
        
        AllHashes=[x.text for x in Current_Page_Txs]

        for x in AllHashes:
            Hash=x  
            Txs_url="https://etherscan.io/tx/"+ Hash
            driver.get(Txs_url)
            try:
                time_element=driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[4]/div/div[2]/span[2]").text
            except:
                MissingVal=MissingVal+1
            else:
                HashData.append(Hash)
                To.append(driver.find_element(By.ID,"contractCopy").text)
                From.append(driver.find_element(By.ID,"addressCopy").text)
                Value.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanValue").text)
                TimeStamp.append(driver.find_element(By.XPATH,"/html/body/div[1]/main/div[3]/div[1]/div[2]/div[1]/div/div[4]/div/div[2]").text)
                TransactionFee.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanTxFee").text)
                GasPrice.append(driver.find_element(By.ID,"ContentPlaceHolder1_spanGasPrice").text)
                TimeData.append(time_element)
            
            #print("  " + HashData[-1]+"  "+TimeData[-1]+"  "+To[-1]+"  "+From[-1]+"  "+Value[-1]+"  "+TimeStamp[-1]+"  "+TransactionFee[-1]+"  "+GasPrice[-1])


        
data_tuples = list(zip(HashData[1:],TimeData[1:],To[1:],From[1:],Value[1:],TimeStamp[1:],TransactionFee[1:],GasPrice[1:]))
df = pd.DataFrame(data_tuples, columns=['Hash','CompletionTime','To','From','Value','TimeStamp','TxnFee','GasPrice'])
df.to_csv('EthDataCSV',index=False)

now = datetime.now().time()
print("End Time =", now)

driver.quit( )    


Start Time = 17:36:03.311938
ENTER STARTING BLOCK NUMBER 14800000
Total Blocks To Fetch1
Total transaction in the block 467
Total Pages 10
End Time = 17:40:44.532931


In [16]:
df


,Hash,CompletionTime,To,From,Value,TimeStamp,TxnFee,GasPrice
0,0x72e0d25c35cdb460e4ede2da05b5c61856054b0ba631...,| Confirmed within 1 min,0xe77f88d9160e7d496d976cd44a54b9038586d157,0xd59391380ddbc47b095613db3f36499a5d987cfc,0 Ether ($0.00),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.001338942788189102 Ether ($1.58),0.000000028730828234 Ether (28.730828234 Gwei)
1,0xd233f07416e05d13983c5c77d953ac206ca24ec5d230...,| Confirmed within 1 min,0xc2e50b16a9f9b2b7048f4291c1c448dcf53873fb,0xf57fe6a662c7ce1f2d89657663531183d65f851f,0.6684481175922594 Ether ($788.93),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.000603347392914 Ether ($0.71),0.000000028730828234 Ether (28.730828234 Gwei)
2,0xab0c301624071a3b43f2e4d05ec2f8b8fed591714dd5...,| Confirmed within 1 min,0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2,zinhimself.eth,0 Ether ($0.00),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.000873876871565344 Ether ($1.03),0.000000028730828234 Ether (28.730828234 Gwei)
3,0xc44946ddfba04a738f43a129154e82ae488f89d2ac76...,| Confirmed within 30 secs,0x2260fac5e5542a773aa44fbcfedf7c193bc2c599,0x5f0bf5bd6facb59bddfe71c8e2dc19b9df54788b,0 Ether ($0.00),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.001425365119516974 Ether ($1.68),0.000000028730828234 Ether (28.730828234 Gwei)
4,0x362cd25b4e70c5696e787aad336c001cffc4745a4249...,| Confirmed within 30 secs,0x7f268357a8c2552623316e2562d90e642bb538e5,0xf826ccfe15d7d27df0fb9d6cf81a5d1a9d05732f,0.012 Ether ($14.16),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.009747163325010372 Ether ($11.50),0.000000028730828234 Ether (28.730828234 Gwei)
...,...,...,...,...,...,...,...,...
236,0xbc32fd3fd9adf645db9d52122c60c85fe393abffe980...,| Confirmed within 30 secs,0x3ab34d31b44075b920e23ab22c8ddeec3370389f,0x3cd751e6b0078be393132286c442345e5dc49699,0.0155 Ether ($18.20),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.000613847392914 Ether ($0.72),0.000000029230828234 Ether (29.230828234 Gwei)
237,0x1f8a1e4ffdfd4648152eeeae498e04aa10a86d9fb977...,| Confirmed within 30 secs,0xa090e606e30bd747d4e6245a1517ebe430f0057e,0xd27bec4b09a97a98ffabc4a740243107c14501c3,0.01666226797292917 Ether ($19.56),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.000613847392914 Ether ($0.72),0.000000029230828234 Ether (29.230828234 Gwei)
238,0xdec34cba8f0e2cc66df71bc2670057cc1b02bc1b1d80...,| Confirmed within 30 secs,0xa090e606e30bd747d4e6245a1517ebe430f0057e,0x98d9def34307d300cf8e8d1ba852e2a636e92d2b,0.645102 Ether ($757.43),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.000613847392914 Ether ($0.72),0.000000029230828234 Ether (29.230828234 Gwei)
239,0xcff553ad379c44920ca2d8a2bfa5a2d5af761e000a63...,| Confirmed within 30 secs,0xa090e606e30bd747d4e6245a1517ebe430f0057e,0x741fd5f97b163d5092f9a1fc1cc63bc5b0e262ec,0.8000100148723508 Ether ($939.31),26 days 19 hrs ago (May-18-2022 04:56:57 PM +U...,0.000613847392914 Ether ($0.72),0.000000029230828234 Ether (29.230828234 Gwei)
